In [1]:
%pip install -r eval_requirements.txt


  Using cached argilla-1.29.1-py3-none-any.whl.metadata (17 kB)
  Using cached wrapt-1.14.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
Using cached argilla-1.29.1-py3-none-any.whl (417 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.6 MB/s eta 0:00:00
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
Using cached wrapt-1.14.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (77 kB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
      Successfully uninstalled wrapt-1.13.3
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
  Attempting uninstall: argilla
    Found existing installation: argilla 2.0.0
    Uninstalling argilla-2.0.0:
    

In [8]:
from datasets import load_dataset
import argilla as rg
import json
import re
from typing import List, Tuple
import json
from utils import compute_argilla_entities
import os
from dotenv import load_dotenv

load_dotenv("../env.txt")

# dataset used
data_path = 'MichaelAI23/English_CVs'

In [10]:
data = load_dataset(data_path)
data["train"]

Dataset({
    features: ['output', 'hobbies', 'personal', 'work_experience', 'skills', 'academia', 'education', 'overall'],
    num_rows: 1000
})

In [11]:
train_val = data["train"].train_test_split(
    test_size=100, shuffle=True, seed=42
)

In [12]:
train_data = train_val["train"]
val_data = train_val["test"]

In [9]:
rg.init(
    api_url=os.getenv("ARGILLA_API_URL"),
    api_key=os.getenv("ARGILLA_API_KEY"),
    workspace="admin"
)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.26.0 using a different client version (1.29.1).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [13]:
CV_text = val_data[0]["overall"]

In [14]:
CV_text

'As a Business Analyst at TechSolutions Inc., I was responsible for analyzing market trends and providing strategic recommendations to drive business growth. I started my role in January 2014 after graduating in 2012 and continued until December 2018, where I successfully implemented data-driven solutions that improved operational efficiency.\n- Published research paper on "The Impact of Data Analytics in Business Decision Making" in Journal of Business Intelligence, 2012\n- Co-authored a study on "Predictive Modeling for Customer Churn Analysis" in International Conference on Business Analytics, 2013\n- Presented findings on "Optimizing Supply Chain Management through Big Data Analysis" at the Annual Meeting of Operations Research Society, 2014\n- Contributed to a book chapter titled "Emerging Trends in Business Forecasting Techniques" published by Springer, 2015\n- Received Best Paper Award for research on "Machine Learning Applications in Marketing Strategy Development" at the Acade

In [15]:
output = val_data[0]["output"]

#### Compare predictions and the labels

In [17]:
with open(f"textmarker_predictions.json", "r", encoding="utf-8") as fp:
    CV_preds = json.load(fp)["model_preds"]

In [18]:
preds = list(CV_preds.values())
preds = [pred[0]["generated_text"] for pred in preds]

In [19]:
preds[0]

"{'pe': {'s': 'Michael', 'e': ' hotel.'}, 'ed': {'s': '', 'e': ''}, 'wo': {'s': 'As', 'e': ' efficiency.'}, 'sk': {'s': 'Experienced', 'e': ' growth.'}}"

In [20]:
labels = val_data["output"]

In [21]:
json.loads(preds[0].replace("\'", "\""))

{'pe': {'s': 'Michael', 'e': ' hotel.'},
 'ed': {'s': '', 'e': ''},
 'wo': {'s': 'As', 'e': ' efficiency.'},
 'sk': {'s': 'Experienced', 'e': ' growth.'}}

In [22]:
json.loads(labels[0].replace("\'", "\"")) == json.loads(preds[0].replace("\'", "\""))

True

In [23]:
sum = 0
for i in range(len(labels)):
    if json.loads(preds[i].replace("\'", "\"")) != json.loads(labels[i].replace("\'", "\"")):
        print(val_data["overall"][i])
        print(preds[i])
        print(labels[i])
        sum += 1
        print(sum)

- Proficient in designing complex technical solutions that align with business objectives and requirements, demonstrating a deep understanding of architecture principles and best practices.
- Skilled in translating high-level business strategies into detailed technical designs, ensuring scalability, security, and performance are considered at every stage of the solution development process.
- Experienced in leading cross-functional teams and collaborating with stakeholders to drive consensus on architectural decisions, fostering a culture of innovation and continuous improvement.
- Proven ability to communicate effectively with both technical and non-technical audiences, presenting complex ideas in a clear and concise manner to facilitate decision-making and project success.
- Co-authored a research paper on "Optimizing Cloud Solutions for Scalability" published in the International Journal of Cloud Computing in 2020.
- Presented findings on "Integration Strategies for Enterprise Syste

In [19]:
sum = 0
for i in range(len(labels)):
    if json.loads(preds[i].replace("\'", "\"")) != json.loads(labels[i].replace("\'", "\"")):
        print(val_data["overall"][i])
        print(preds[i])
        print(labels[i])
        sum += 1
        print(sum)

- Proficient in designing complex technical solutions that align with business objectives and requirements, demonstrating a deep understanding of architecture principles and best practices.
- Skilled in translating high-level business strategies into detailed technical designs, ensuring scalability, security, and performance are considered at every stage of the solution development process.
- Experienced in leading cross-functional teams and collaborating with stakeholders to drive consensus on architectural decisions, fostering a culture of innovation and continuous improvement.
- Proven ability to communicate effectively with both technical and non-technical audiences, presenting complex ideas in a clear and concise manner to facilitate decision-making and project success.
- Co-authored a research paper on "Optimizing Cloud Solutions for Scalability" published in the International Journal of Cloud Computing in 2020.
- Presented findings on "Integration Strategies for Enterprise Syste

In [17]:
sum = 0
for i in range(len(labels)):
    if preds[i] != labels[i]:
        print(val_data["overall"][i])
        print(preds[i])
        print(labels[i])
        sum += 1
        print(sum)

- Proficient in designing complex technical solutions that align with business objectives and requirements, demonstrating a deep understanding of architecture principles and best practices.
- Skilled in translating high-level business strategies into detailed technical designs, ensuring scalability, security, and performance are considered at every stage of the solution development process.
- Experienced in leading cross-functional teams and collaborating with stakeholders to drive consensus on architectural decisions, fostering a culture of innovation and continuous improvement.
- Proven ability to communicate effectively with both technical and non-technical audiences, presenting complex ideas in a clear and concise manner to facilitate decision-making and project success.
- Co-authored a research paper on "Optimizing Cloud Solutions for Scalability" published in the International Journal of Cloud Computing in 2020.
- Presented findings on "Integration Strategies for Enterprise Syste

In [24]:
texts = val_data["overall"]
labels = val_data["output"]

In [25]:
labels[27]

"{'pe': {'s': 'Penelope', 'e': ' reservations.'}, 'ed': {'s': '- Master', 'e': ', 2018'}, 'wo': {'s': '\\nAssociate', 'e': ' supervision.'}, 'sk': {'s': '- Proficient', 'e': ' precision.'}}"

In [26]:
preds_filtered = [pred for idx, pred in enumerate(preds) if pred != labels[idx]]
labels_filtered = [label for idx, label in enumerate(labels) if label != preds[idx]]
texts_filtered = [txt for idx, txt in enumerate(texts) if preds[idx] != labels[idx]]

In [27]:
labels_filtered[5]

"{'pe': {'s': 'Caleb', 'e': ' style.'}, 'ed': {'s': '', 'e': ''}, 'wo': {'s': 'Solution', 'e': ' initiatives.'}, 'sk': {'s': '- Proficient', 'e': ' timelines.'}}"

In [28]:
preds_filtered[5]

"{'pe': {'s': 'Caleb', 'e': ' style.'}, 'ed': {'s': '', 'e': ''}, 'wo': {'s': 'Solution', 'e': ' timelines.'}, 'sk': {'s': '- Proficient', 'e': ' timelines.'}}"

In [29]:
texts_filtered[5]

'- Proficient in designing complex technical solutions that align with business objectives, leveraging a deep understanding of cloud technologies and architecture patterns.\n- Skilled at translating high-level requirements into detailed design specifications, ensuring scalability, reliability, and security of the solution.\n- Experienced in collaborating with cross-functional teams to drive successful implementation and delivery of projects within tight timelines.\nCaleb Gonçalves can be reached at caleb.goncalves@email.com for any inquiries. He resides at 123 Palm Avenue, Mumbai, India and his phone number is +91-9876543210. Caleb Gonçalves is a distinguished guest at The St. Regis in Mumbai, known for his impeccable taste and style.\nSolution Architect, 2014 - Present at Tech Solutions Inc.\n- Design and implement innovative solutions for clients in various industries to address their complex business challenges.\n- Collaborate with cross-functional teams to ensure the successful del

In [30]:
token_dataset = compute_argilla_entities(preds, labels, texts, "LLM_Textmarker_mistral_7B_02_JsonFormer")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [31]:
for record in token_dataset:
    rg.log(record, name="llm_textmarker_with_preds")

Output()

1 records logged to ]8;id=995772;https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds\https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds]8;;\

Output()

1 records logged to ]8;id=491962;https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds\https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds]8;;\

Output()

...

1 records logged to ]8;id=631754;https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds\https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds]8;;\

Output()

1 records logged to ]8;id=160604;https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds\https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds]8;;\

Output()

1 records logged to ]8;id=719355;https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds\https://michaelai23-argillatest.hf.space/datasets/admin/llm_textmarker_with_preds]8;;\

In [33]:
# Load the dataset
dataset = rg.load(name="llm_textmarker_with_preds", workspace="admin")

In [74]:
# rg.delete_records(name="llm_textmarker_with_preds")

(100, 100)